In [1]:
from timeit import default_timer as timer
from datetime import timedelta

import time
import warnings
from ripser import ripser
import numpy as np
from numpy import inf
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os


from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import umap

import pandas as pd

from scipy.spatial.distance import pdist, squareform
import stablerank.srank as sr

warnings.filterwarnings('ignore')

In [2]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EbHRGswx8pxPtp1ZDF1l28YB_qXGIAX7qzbn5II29kLjGA?download=1 -O sonar.csv

--2024-01-29 02:30:51--  https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EbHRGswx8pxPtp1ZDF1l28YB_qXGIAX7qzbn5II29kLjGA?download=1
Resolving unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)... 13.107.138.10, 13.107.136.10
Connecting to unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)|13.107.138.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/adines_unirioja_es/Documents/sonar.csv?ga=1 [following]
--2024-01-29 02:30:52--  https://unirioja-my.sharepoint.com/personal/adines_unirioja_es/Documents/sonar.csv?ga=1
Reusing existing connection to unirioja-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [application/octet-stream]
Saving to: 'sonar.csv'

     0K .......... .......... .......... .......... .......... 58% 14,7M 0s
    50K .......... .......... .......... .....                100% 44,3M=0,004s

2024-01-29 02:30:52 (20,4 MB/s) - 'sonar.csv' saved [87776/87776]



In [3]:
def load_prima():
  df=pd.read_csv("sonar.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [4]:
def preparar_dataset(data,target,n_labeled=25):
  unos=np.where(target==1)[0]
  ceros=np.where(target==0)[0]

  datos_uno=np.array(data)[unos]
  datos_cero=np.array(data)[ceros]

  unos_valid=int((len(list(unos))-n_labeled)*0.2)
  ceros_valid=int((len(list(ceros))-n_labeled)*0.2)

  random.seed(15)
  indices_unos=random.sample(list(unos),n_labeled+unos_valid)

  random.seed(10)
  indices_ceros=random.sample(list(ceros),n_labeled+ceros_valid)


  puntos_unos=np.array(data)[indices_unos[0:n_labeled]]
  puntos_ceros=np.array(data)[indices_ceros[0:n_labeled]]

  puntos_unos_valid=np.array(data)[indices_unos[n_labeled:]]
  puntos_ceros_valid=np.array(data)[indices_ceros[n_labeled:]]

  unlabeled_unos=list(set(unos)-set(indices_unos))
  unlabeled_ceros=list(set(ceros)-set(indices_ceros))
  unlabeled=unlabeled_unos+unlabeled_ceros
  X_unlabeled_unos=np.array(data)[unlabeled_unos]
  X_unlabeled_ceros=np.array(data)[unlabeled_ceros]

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [5]:
def analizar_punto_stable_rank(punto,puntos_ceros,puntos_unos,mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10,distributions,number_instances,sample_size,contour=sr.standard_contour(),clustering_method="complete",w_p=inf,w_q=1,reduced=True,p=2):
    (mh0_00_p,mh0_11_p,mh0_01_p,mh0_10_p,mh1_00_p,mh1_11_p,mh1_01_p,mh1_10_p)=calcular_sranks_punto_v_muestra(punto,puntos_ceros,puntos_unos,distributions,number_instances,sample_size,contour=sr.standard_contour(),clustering_method="complete",w_p=inf,w_q=1,reduced=True)
    
    
    distance_cero=max(abs(mh0_00.lp_distance(mh0_00_p,p)),abs(mh0_10.lp_distance(mh0_10_p,p)))
    distance_uno=max(abs(mh0_11.lp_distance(mh0_11_p,p)),abs(mh0_01.lp_distance(mh0_01_p,p)))

        
    if distance_cero>distance_uno:
        clase=1
    else:
        clase=0
    return clase

In [6]:
def calcular_sranks_muestra(puntos_ceros,puntos_unos,sd_distribution,number_instances,sample_size,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True):
    
    n_ceros=len(puntos_ceros)
    n_unos=len(puntos_unos)
    distribution=sr.get_distribution(name="norm", loc=0, scale=sd_distribution)
    
    ceros_distances=squareform(pdist(puntos_ceros,"euclidean"))
    distance_ceros=sr.Distance(ceros_distances)
    
    unos_distances=squareform(pdist(puntos_unos,"euclidean"))
    distance_unos=sr.Distance(unos_distances)
    
    ceros_wr_unos_distances=[]
    for point in puntos_unos:
        d_ceros_mod = squareform(pdist(np.array([point.tolist()]+puntos_ceros.tolist()), "euclidean"))
        l=d_ceros_mod[0].tolist()
        l.pop(0)
        ceros_wr_unos_distances.append(l)
    ceros_wr_unos_distances=np.array(ceros_wr_unos_distances)
    
    unos_wr_ceros_distances=[]
    for point in puntos_ceros:
        d_unos_mod = squareform(pdist(np.array([point.tolist()]+puntos_unos.tolist()), "euclidean"))
        l=d_unos_mod[0].tolist()
        l.pop(0)
        unos_wr_ceros_distances.append(l)
    unos_wr_ceros_distances=np.array(unos_wr_ceros_distances)
    
    h0_sr_00 = []
    h1_sr_00 = []
    h0_sr_11 = []
    h1_sr_11 = []
    h0_sr_01 = []
    h1_sr_01 = []
    h0_sr_10 = []
    h1_sr_10 = []
    mh0_00 = 0
    mh1_00 = 0
    mh0_11 = 0
    mh1_11 = 0
    mh0_01 = 0
    mh1_01 = 0
    mh0_10 = 0
    mh1_10 = 0
    for point in ceros_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_ceros.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_ceros.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_00.append(f)
        h1_sr_00.append(g)
        mh0_00 = mh0_00+f
        mh1_00 = mh1_00+g

    for point in unos_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_unos.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_unos.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_11.append(f)
        h1_sr_11.append(g)
        mh0_11 = mh0_11+f
        mh1_11 = mh1_11+g

    for point in ceros_wr_unos_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_ceros.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_ceros.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_01.append(f)
        h1_sr_01.append(g)
        mh0_01 = mh0_01+f
        mh1_01 = mh1_01+g

    for point in unos_wr_ceros_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_unos.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_unos.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_10.append(f)
        h1_sr_10.append(g)
        mh0_10 = mh0_10+f
        mh1_10 = mh1_10+g

    
    mh0_00 = mh0_00/n_ceros
    mh1_00 = mh1_00/n_ceros
    mh0_11 = mh0_11/n_unos
    mh1_11 = mh1_11/n_unos
    mh0_01 = mh0_01/n_ceros
    mh1_01 = mh1_01/n_ceros
    mh0_10 = mh0_10/n_unos
    mh1_10 = mh1_10/n_unos
    
    return (mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10)

In [7]:
def calcular_sranks_punto_v_muestra(punto,puntos_ceros,puntos_unos,sd_distribution,number_instances,sample_size,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True):
    
    n_ceros=len(puntos_ceros)
    n_unos=len(puntos_unos)
    distribution=sr.get_distribution(name="norm", loc=0, scale=sd_distribution)
    
    ceros_distances=squareform(pdist(puntos_ceros,"euclidean"))
    distance_ceros=sr.Distance(ceros_distances)
    
    unos_distances=squareform(pdist(puntos_unos,"euclidean"))
    distance_unos=sr.Distance(unos_distances)
    
    ceros_wr_punto_distances=[]
    d_ceros_mod = squareform(pdist(np.array([punto.tolist()]+puntos_ceros.tolist()), "euclidean"))
    l=d_ceros_mod[0].tolist()
    l.pop(0)
    ceros_wr_punto_distances.append(l)
    ceros_wr_punto_distances=np.array(ceros_wr_punto_distances)
    
    unos_wr_punto_distances=[]
    d_unos_mod = squareform(pdist(np.array([punto.tolist()]+puntos_unos.tolist()), "euclidean"))
    l=d_unos_mod[0].tolist()
    l.pop(0)
    unos_wr_punto_distances.append(l)
    unos_wr_punto_distances=np.array(unos_wr_punto_distances)
    
    distance_ceros_mod=sr.Distance(d_ceros_mod)
    distance_unos_mod=sr.Distance(d_unos_mod)
    
    h0_sr_00 = []
    h1_sr_00 = []
    h0_sr_11 = []
    h1_sr_11 = []
    h0_sr_01 = []
    h1_sr_01 = []
    h0_sr_10 = []
    h1_sr_10 = []
    mh0_00 = 0
    mh1_00 = 0
    mh0_11 = 0
    mh1_11 = 0
    mh0_01 = 0
    mh1_01 = 0
    mh0_10 = 0
    mh1_10 = 0
    for point in ceros_wr_punto_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_ceros_mod.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_ceros_mod.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_00.append(f)
        h1_sr_00.append(g)
        mh0_00 = f
        mh1_00 = g

    for point in unos_wr_punto_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_unos_mod.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_unos_mod.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_11.append(f)
        h1_sr_11.append(g)
        mh0_11 = f
        mh1_11 = g

    for point in ceros_wr_punto_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_ceros.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_ceros.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_01.append(f)
        h1_sr_01.append(g)
        mh0_01 = f
        mh1_01 = g

    for point in unos_wr_punto_distances:
        p = distribution(point)
        s = sr.get_sample(number_instances, sample_size, p)
        f = distance_unos.get_h0sr(sample=s,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True)
        b = distance_unos.get_bc(sample=s, maxdim=1)
        g = sr.bc_to_sr(b,degree="H1")
        h0_sr_10.append(f)
        h1_sr_10.append(g)
        mh0_10 = f
        mh1_10 = g
        

    
    return (mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10)

In [8]:
def analizar_puntos(data,target,sd_distribution,number_instances,sample_size,contour=sr.standard_contour(),w_p=inf,w_q=1,reduced=True,p=2,th=0,clustering_method="single",reduccion=False):
    (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
    embedding=data
    if reduccion==True:
        embedding=umap.UMAP(random_state=75).fit_transform(data)
    (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


    bien=0
    mal=0
    dudoso=0

    puntos_ceros_final=puntos_ceros.tolist()
    puntos_unos_final=puntos_unos.tolist()
    puntos_dudosos_final=[]

    (mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10)=calcular_sranks_muestra(puntos_ceros_umap,puntos_unos_umap,sd_distribution,number_instances,sample_size,contour,clustering_method,w_p,w_q,reduced)

    

    for i,punto in enumerate(X_unlabeled_unos_umap):
        clase=analizar_punto_stable_rank(punto,puntos_ceros_umap,puntos_unos_umap,mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10,sd_distribution,number_instances,sample_size,contour,clustering_method,w_p,w_q,reduced,p)
        if clase==1:
          bien=bien+1
          puntos_unos_final.append(X_unlabeled_unos[i])
        elif clase==0:
          mal=mal+1
          puntos_ceros_final.append(X_unlabeled_unos[i])
        else:
          dudoso=dudoso+1
          puntos_dudosos_final.append(X_unlabeled_unos[i])



    for i,punto in enumerate(X_unlabeled_ceros_umap):
        clase=analizar_punto_stable_rank(punto,puntos_ceros_umap,puntos_unos_umap,mh0_00,mh0_11,mh0_01,mh0_10,mh1_00,mh1_11,mh1_01,mh1_10,sd_distribution,number_instances,sample_size,contour,clustering_method,w_p,w_q,reduced,p)
        if clase==1:
          mal=mal+1
          puntos_unos_final.append(X_unlabeled_ceros[i])
        elif clase==0:
          bien=bien+1
          puntos_ceros_final.append(X_unlabeled_ceros[i])
        else:
          dudoso=dudoso+1
          puntos_dudosos_final.append(X_unlabeled_ceros[i])

    comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

    return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [9]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90)
    modelSVMLineal.fit(datos, labels)

    mal=np.count_nonzero(modelSVMLineal.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelSVMLineal.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("--------------Clasificador SVM--------------")
    resumen(total-mal,mal,0)
    print("")


    # datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    # labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)

    mal=np.count_nonzero(modelRandomForest.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelRandomForest.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("-------------- Clasificador Random Forest--------------")
    resumen(total-mal,mal,0)
    print("")

In [10]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Bien anotados '+str(bien))
  print('Mal anotados '+str(mal))
  print('Sin anotar '+str(dudosos))
  print('')
  if bien+mal!=0:
    print('Porcentaje correcto '+str(bien/(bien+mal)))
  else:
    print('Porcentaje correcto 0')
  print('Porcentaje anotado '+str((bien+mal)/(bien+mal+dudosos)))

# Standard unreduced stable rank

## No threshold

In [11]:
(data,target) = load_prima()


In [12]:
l1=min(data[:,0])
l2=min(data[:,1])
l3=max(data[:,0])
l4=max(data[:,1])

s1=((l3-l1)/2+(l4-l2)/2)/2
s1
s2=s1/2
s3=1.5*s1
s4=2*s1
s5=0.1

number_instances=100
sample_size=4

In [14]:
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,s2,number_instances,sample_size)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 21
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.6774193548387096
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 15
Mal anotados 16
Sin anotar 0

Porcentaje correcto 0.4838709677419355
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 67
Mal anotados 60
Sin anotar 0

Porcentaje correcto 0.5275590551181102
Porcentaje anotado 1.0
